In [8]:
%matplotlib inline  
import matplotlib.pyplot as plt   
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook
from time import sleep
import scipy
import operator
import difflib
import math

from IPython.core.display import display,HTML
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser



try:
    import cPickle as pickle  ##################################
   #it is faster than pickle!
except:
    import pickle
    
import unicodedata
import networkx as nx
import itertools
import seaborn as sns   ### https://seaborn.pydata.org/tutorial/categorical.html
import time  



import plotly.plotly as py


# i only need my credentials if i want to plot online --- and send plots to server (limits per day apply!)
#import plotly.tools as tls
#tls.set_credentials_file(username='juliettapc', api_key='deyNIvtOoDZ5PLmrHlhd')  # my plotly account credentials


import pygraphviz
from networkx.drawing.nx_agraph import graphviz_layout




########## to be able to plot offline (without sending the plots to the plotly server every time)
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
################



##### for getting geolocation data  and to calculate distance between two geolocations
import requests
import json
import geopy.distance   




In [ ]:
# 

In [9]:
# path = "/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Dropbox/"
# input_file = 'Dropbox_datafile_complete_new_ranking.csv'
# #input_file = 'Dropbox_datafile_may22_2017_modified_added_univ_country_geolocation_RUCC_population_density_when_available_career_stage_GINI_folder_act_categories.csv'
# df = pd.read_csv(path+input_file, sep=';',na_values=["NAN","-1","null"],low_memory=False, parse_dates=['folder_creation_date','date_last_change']) # set header=0 if i wanna pass it my own list of header names
# df.drop('Unnamed: 0', axis=1, inplace=True)
# print df.shape   #  1403349, 38 



# ############  load the aggregated network
# pickle_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/Networks/network_all_new.pickle'
# file = open(pickle_name,'r')
# G_no_filter = pickle.load(file)
# print "\n",pickle_name, len (G_no_filter.nodes()), len(G_no_filter.edges())    #   440353 3659098




############  load the aggregated network   filtering links by activity (if a node's activity =0, then it is not link to any other members of a folder)
pickle_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/Networks/network_all_weighted_by_act_thresh1.pickle'
file = open(pickle_name,'r')
G_activity = pickle.load(file)
print "\n",pickle_name, len (G_activity.nodes()), len(G_activity.edges())    #   440353 3659098






# ######## load dictionary aggregated info by user
pickle_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_user_id_user_attr_more_var_.pickle'
with open(pickle_name, 'rb') as handle:
     dict_user_id_user_attr = pickle.load(handle)
print pickle_name, len(dict_user_id_user_attr.keys())




/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/Networks/network_all_weighted_by_act_thresh1.pickle 440353 450589
/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_user_id_user_attr_more_var_.pickle 440353


In [ ]:
# dict_user_id_user_attr
# 378535942: {'CC_filtered_by_act': 0.0,
#   'CC_unfiltered': 1.0,
#   'SD_fract_work_over_diff_folders': 0.0,
#   'avg_fract_work_over_diff_folders': 0.0,
#   'career_stage': nan,
#   'category_total_cit': nan,
#   'category_total_last_auth': nan,
#   'category_total_publ': nan,
#   'k_filtered_by_act': 0,
#   'k_unfiltered': 11,
#   'kshell_filtered_by_act': 0,
#   'kshell_unfiltered': 11,did you make yours too
#   'list_folders': [803186458],
#   'mean_folder_lifespan': 1.0,
#   'number_folders': 1,
#   'user_tot_act': 0.0,
#   'user_tot_num_adds': 0.0,
#   'user_tot_num_deletes': 0.0,
#   'user_tot_num_edits': 0.0,
#   'user_univ_ranking': 281.0},

In [ ]:

# #### if the dict is larger than G.nodes(), and the set_node_attribute does not work in that case, so i get a selection of the dict only with the common nodes:
# new_dict_user_id = {k: dict_user_id_domain[k] for k in set(G.nodes()) & set(dict_user_id_domain.keys())}
    
# # i add the email as node attributes    
# nx.set_node_attributes(G, 'email', new_dict_user_id)
# ############




In [ ]:
G= G_activity  ## or G_activity

dict_node_degree={}
dict_node_number_folders={}
dict_node_user_tot_act={}
dict_node_user_univ_ranking={}
dict_node_user_career_stage={}
dict_node_user_avg_fract_work_over_diff_folders={}
dict_node_category_total_cit={}
dict_node_category_total_publ={}
dict_node_kshell_filtered_by_act={}
dict_node_mean_folder_lifespan={}

for user_id in dict_user_id_user_attr:
    
    dict_node_degree[user_id]=G.degree(user_id)
    dict_node_number_folders[user_id]=dict_user_id_user_attr[user_id]['number_folders']
    dict_node_user_tot_act[user_id]=dict_user_id_user_attr[user_id]['user_tot_act']
    dict_node_user_univ_ranking[user_id]=dict_user_id_user_attr[user_id]['user_univ_ranking']
    dict_node_user_career_stage[user_id]=dict_user_id_user_attr[user_id]['career_stage']
    dict_node_user_avg_fract_work_over_diff_folders[user_id]=dict_user_id_user_attr[user_id]['avg_fract_work_over_diff_folders']
    dict_node_category_total_cit[user_id]=dict_user_id_user_attr[user_id]['category_total_cit']
    dict_node_category_total_publ[user_id]=dict_user_id_user_attr[user_id]['category_total_publ']
    dict_node_kshell_filtered_by_act[user_id]=dict_user_id_user_attr[user_id]['kshell_filtered_by_act']
    dict_node_mean_folder_lifespan[user_id]=dict_user_id_user_attr[user_id]['mean_folder_lifespan']
    
 

nx.set_node_attributes(G, 'k', dict_node_degree)
nx.set_node_attributes(G, 'number_folders', dict_node_number_folders)
nx.set_node_attributes(G, 'user_tot_act', dict_node_user_tot_act)
nx.set_node_attributes(G, 'user_univ_ranking', dict_node_user_univ_ranking)
nx.set_node_attributes(G, 'career_stage', dict_node_user_career_stage)
nx.set_node_attributes(G, 'user_avg_fract_work_over_diff_folders', dict_node_user_avg_fract_work_over_diff_folders)
nx.set_node_attributes(G, 'category_total_cit', dict_node_category_total_cit)
nx.set_node_attributes(G, 'category_total_publ', dict_node_category_total_publ)
nx.set_node_attributes(G, 'kshell_filtered_by_act', dict_node_kshell_filtered_by_act)
nx.set_node_attributes(G, 'mean_folder_lifespan', dict_node_mean_folder_lifespan)



print "done"

done


In [ ]:
G.nodes(data=True)

In [ ]:


list_var=['k', 'number_folders', 'user_tot_act', 'user_univ_ranking',  'career_stage',  'user_avg_fract_work_over_diff_folders',  'category_total_cit',  'category_total_publ',  'kshell_filtered_by_act', 'mean_folder_lifespan']
for var in list_var:
    print var

    list_nodes=[]
    for node in G.nodes(): ### i need to create lists of nodes for each variable, so i only include the nodes that have a non-nan value for that variable
        value=G.node[node][var]
        #print node, value
        
        try:
            int(value)  # BECAUSE NAN IS A FLOAT, BUT IT CAN NOT BE CONVERTED TO INT
            list_nodes.append(node)
        except: pass
            #print "no cant do"
    print " len of list:", len(list_nodes)
    H = G.subgraph(list_nodes)
    print " Size of subgraph:",len(H.nodes()),"nodes   ",len(H.edges()),"edges   "
    print " assortativity", nx.attribute_assortativity_coefficient(H,var)
    print 
    
    
    

k
 len of list: 440353
 Size of subgraph: 440353 nodes    450589 edges   
 assortativity 0.373837539772

number_folders
 len of list: 440353
 Size of subgraph: 440353 nodes    450589 edges   
 assortativity 0.103194172983

user_tot_act
 len of list: 440353
 Size of subgraph: 440353 nodes    450589 edges   
 assortativity

In [ ]:
# df[df['user_id'] == 51380975]
# df[df['folder_id'] == 216959429]

In [ ]:
# G= nx.Graph()
# G.add_nodes_from([0,1,2,3,4],color='red')
# G.add_nodes_from([5,6,7,8,9],color='blue')
# G.add_edges_from([(0,9),(2,3),(3,4),(5,6),(7,8),(8,9)])
# print(nx.attribute_assortativity_coefficient(G,'color'))
